In [1]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from itertools import chain

In [2]:
data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd'
model_dir = 'checkpoint/model'

In [3]:
word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
word_vocab_size = 400001
pos_vocab_size = 10
dep_vocab_size = 21
relation_classes = 19
word_state_size = 100
other_state_size = 100
batch_size = 10
channels = 3
lambda_l2 = 0.0001
max_len_path = 10

In [4]:
with tf.name_scope("input"):
    path_length = tf.placeholder(tf.int32, shape=[2, batch_size], name="path1_length")
    word_ids = tf.placeholder(tf.int32, shape=[2, batch_size, max_len_path], name="word_ids")
    pos_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="pos_ids")
    dep_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="dep_ids")
    y = tf.placeholder(tf.int32, [batch_size], name="y")

In [5]:
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

In [6]:
with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

In [7]:
with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [8]:
word_hidden_state = tf.zeros([batch_size, word_state_size], name='word_hidden_state')
word_cell_state = tf.zeros([batch_size, word_state_size], name='word_cell_state')
word_init_state = tf.contrib.rnn.LSTMStateTuple(word_hidden_state, word_cell_state)

In [9]:
other_hidden_states = tf.zeros([channels-1, batch_size, other_state_size], name="hidden_state")
other_cell_states = tf.zeros([channels-1, batch_size, other_state_size], name="cell_state")

In [10]:
other_init_states = [tf.contrib.rnn.LSTMStateTuple(other_hidden_states[i], other_cell_states[i]) for i in range(channels-1)]

In [11]:
with tf.variable_scope("word_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[0], sequence_length=path_length[0], initial_state=word_init_state)
    state_series_word1 = tf.reduce_max(state_series, axis=1)

In [12]:
with tf.variable_scope("word_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[1], sequence_length=path_length[1], initial_state=word_init_state)
    state_series_word2 = tf.reduce_max(state_series, axis=1)

In [13]:
with tf.variable_scope("pos_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[0], sequence_length=path_length[0],initial_state=other_init_states[0])
    state_series_pos1 = tf.reduce_max(state_series, axis=1)

In [14]:
with tf.variable_scope("pos_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[1], sequence_length=path_length[1],initial_state=other_init_states[0])
    state_series_pos2 = tf.reduce_max(state_series, axis=1)

In [15]:
with tf.variable_scope("dep_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[0], sequence_length=path_length[0], initial_state=other_init_states[1])
    state_series_dep1 = tf.reduce_max(state_series, axis=1)

In [16]:
with tf.variable_scope("dep_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[1], sequence_length=path_length[1], initial_state=other_init_states[1])
    state_series_dep2 = tf.reduce_max(state_series, axis=1)

In [17]:
state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

In [18]:
state_series = tf.concat([state_series1, state_series2], 1)

In [19]:
state_series

<tf.Tensor 'concat_2:0' shape=(10, 600) dtype=float32>

In [20]:
with tf.name_scope("hidden_layer"):
    W = tf.Variable(tf.truncated_normal([600, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = tf.matmul(state_series, W) + b

In [21]:
with tf.name_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_hidden_layer, W) + b
    predictions = tf.argmax(logits, 1)

In [22]:
global_step = tf.Variable(0, name="global_step")

In [23]:
tv_all = tf.trainable_variables()
tv_all

[<tf.Variable 'word_embedding/W:0' shape=(400001, 100) dtype=float32_ref>,
 <tf.Variable 'pos_embedding/W:0' shape=(10, 25) dtype=float32_ref>,
 <tf.Variable 'dep_embedding/W:0' shape=(21, 25) dtype=float32_ref>,
 <tf.Variable 'word_lstm1/rnn/basic_lstm_cell/weights:0' shape=(200, 400) dtype=float32_ref>,
 <tf.Variable 'word_lstm1/rnn/basic_lstm_cell/biases:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'word_lstm2/rnn/basic_lstm_cell/weights:0' shape=(200, 400) dtype=float32_ref>,
 <tf.Variable 'word_lstm2/rnn/basic_lstm_cell/biases:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'pos_lstm1/rnn/basic_lstm_cell/weights:0' shape=(125, 400) dtype=float32_ref>,
 <tf.Variable 'pos_lstm1/rnn/basic_lstm_cell/biases:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'pos_lstm2/rnn/basic_lstm_cell/weights:0' shape=(125, 400) dtype=float32_ref>,
 <tf.Variable 'pos_lstm2/rnn/basic_lstm_cell/biases:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'dep_lstm1/rnn/basic_lstm_cell/weights:0' sha

In [27]:
tv_all = tf.trainable_variables()
tv_regu = []
non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0',"global_step:0",'hidden_layer/b:0','softmax_layer/b:0']
for t in tv_all:
    if t.name not in non_reg:
        if(t.name.find('biases')==-1):
            tv_regu.append(t)


In [28]:
tv_regu

[<tf.Variable 'word_lstm1/rnn/basic_lstm_cell/weights:0' shape=(200, 400) dtype=float32_ref>,
 <tf.Variable 'word_lstm2/rnn/basic_lstm_cell/weights:0' shape=(200, 400) dtype=float32_ref>,
 <tf.Variable 'pos_lstm1/rnn/basic_lstm_cell/weights:0' shape=(125, 400) dtype=float32_ref>,
 <tf.Variable 'pos_lstm2/rnn/basic_lstm_cell/weights:0' shape=(125, 400) dtype=float32_ref>,
 <tf.Variable 'dep_lstm1/rnn/basic_lstm_cell/weights:0' shape=(125, 400) dtype=float32_ref>,
 <tf.Variable 'dep_lstm2/rnn/basic_lstm_cell/weights:0' shape=(125, 400) dtype=float32_ref>,
 <tf.Variable 'hidden_layer/W:0' shape=(600, 100) dtype=float32_ref>,
 <tf.Variable 'softmax_layer/W:0' shape=(100, 19) dtype=float32_ref>]

In [29]:
with tf.name_scope("loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    total_loss = loss + l2_loss

In [30]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)

In [31]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

In [32]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()

In [33]:
word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

In [34]:
unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

In [35]:
f = open('train_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

In [36]:
relations = []
for line in open('data/train_relation.txt'):
    relations.append(line.strip().split()[1])

In [37]:
num_batches = int(8000/batch_size)
num_batches

800

In [38]:
pos_tags_vocab = []
for line in open('data/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

In [39]:
dep_vocab = []
for line in open('data/dependency_types.txt'):
    dep_vocab.append(line.strip())

In [40]:
relation_vocab = []
for line in open('data/relation_types.txt'):
    relation_vocab.append(line.strip())

In [41]:
rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

In [42]:
pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

In [43]:
dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

In [44]:
pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

In [45]:
dep2id['OTH'] = 20
id2dep[20] = 'OTH'

In [46]:
JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

In [47]:
 def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [48]:
dep = []
for i in range(8000):
    dep.append(dep_p1[i])

In [49]:
for i in range(8000):
    dep.append(dep_p2[i])

In [50]:
dep_freq = nltk.FreqDist(chain(*dep))
dep_types = dep_freq.most_common(20)

In [51]:
for i in range(8000):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

In [52]:
word_p1_ids = np.ones([8000, max_len_path],dtype=int)
word_p2_ids = np.ones([8000, max_len_path],dtype=int)
pos_p1_ids = np.ones([8000, max_len_path],dtype=int)
pos_p2_ids = np.ones([8000, max_len_path],dtype=int)
dep_p1_ids = np.ones([8000, max_len_path],dtype=int)
dep_p2_ids = np.ones([8000, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

In [53]:
for i in range(8000):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [54]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [55]:
# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [56]:
latest_embd = tf.train.latest_checkpoint(word_embd_dir)
word_embedding_saver.restore(sess, latest_embd)

INFO:tensorflow:Restoring parameters from checkpoint/word_embd/word_embd


In [65]:
feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}

In [66]:
_, t = sess.run([optimizer, total_loss], feed_dict)

In [67]:
t

5.1904964

In [ ]:
num_epochs = 1
for i in range(num_epochs):
    for j in range(num_batches):
        path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
        word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
        pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
        dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
        y_dict = rel_ids[j*batch_size:(j+1)*batch_size]
        
        feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}
        _, loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        print("Step:", step, "loss:",loss)

Step: 4 loss: 3.28352
Step: 5 loss: 3.4505
Step: 6 loss: 3.91823
Step: 7 loss: 3.69251
Step: 8 loss: 2.8949
Step: 9 loss: 2.50026
Step: 10 loss: 3.33039
Step: 11 loss: 2.85601
Step: 12 loss: 2.70937
Step: 13 loss: 3.28743
Step: 14 loss: 2.67206
Step: 15 loss: 3.16435
Step: 16 loss: 2.96526
Step: 17 loss: 3.19775
Step: 18 loss: 2.6358
Step: 19 loss: 3.16826
Step: 20 loss: 3.01632
Step: 21 loss: 2.90241
Step: 22 loss: 2.80949
Step: 23 loss: 3.01437
Step: 24 loss: 2.74901
Step: 25 loss: 2.84607
Step: 26 loss: 2.52782
Step: 27 loss: 3.15327
Step: 28 loss: 2.49627
Step: 29 loss: 2.84882
Step: 30 loss: 2.8855
Step: 31 loss: 2.66097
Step: 32 loss: 2.42558
Step: 33 loss: 2.66786
Step: 34 loss: 2.88996
Step: 35 loss: 2.86373
Step: 36 loss: 3.09539
Step: 37 loss: 2.66648
Step: 38 loss: 2.7805
Step: 39 loss: 2.71029
Step: 40 loss: 2.76583
Step: 41 loss: 2.9642
Step: 42 loss: 2.71039
Step: 43 loss: 2.80893
Step: 44 loss: 2.60541
Step: 45 loss: 2.96271
Step: 46 loss: 2.42622
Step: 47 loss: 2.79787
